In [ ]:
import yfinance as yf
from datetime import timedelta, date, datetime
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import pytz

## 1. Daily data extraction with yahoo finance

In [ ]:
tickers = ['SPY', 'IWM', 'VXUS', 'AAXJ', 'EEM', 'QQQ', 'GLD', 'AGG', 'BNDX', 'VNQ']

In [ ]:
def download_data_1d(tickers, end_date, start_date):
    print(tickers)
    print(end_date)
    print(start_date)

    # Convert dates to datetime objects with specific timezone
    tz = pytz.timezone('America/New_York')
    end_date_tz = tz.localize(pd.Timestamp(end_date))
    start_date_tz = tz.localize(pd.Timestamp(start_date))

    # Initialize the resulting DataFrame
    result = pd.DataFrame()

    # Download data for each ticker
    for i, ticker in enumerate(tqdm(tickers, desc="Downloading data"), start=1):
        try:
            data = yf.download(tickers=ticker, start=start_date, end=end_date, interval="1d")
            data['tic'] = ticker
            data['datadate'] = data.index
            result = pd.concat([result, data], axis=0)
        except Exception as e:
            print(f"Error downloading data for {ticker}: {str(e)}")
            continue

    result.reset_index(inplace=True)
    result['datadate'] = result['datadate'].dt.date
    result = result.drop(columns=['Date'], axis=1)
    return result

In [ ]:
result = download_data_1d(tickers, '2023-12-31', '2013-10-31')

In [ ]:
# Add info of the dividends
filtered_dividends_total = pd.DataFrame()

for ticker in tickers:
    etf = yf.Ticker(ticker)

    dividends = etf.dividends
    filtered_dividends = dividends["2013-10-31":"2023-12-31"]

    filtered_dividends = filtered_dividends.to_frame(name="Dividends")
    filtered_dividends["tic"] = ticker
    filtered_dividends['datadate'] = filtered_dividends.index
    filtered_dividends_total = pd.concat([filtered_dividends_total, filtered_dividends])

filtered_dividends_total.reset_index(inplace=True)
filtered_dividends_total['datadate'] = filtered_dividends_total['datadate'].dt.date
filtered_dividends_total = filtered_dividends_total.drop(columns=['Date'], axis=1)

In [ ]:
date_info = pd.merge(result, filtered_dividends_total, on = ['datadate', 'tic'], how = 'left')
new_column_order = ['datadate', 'tic', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends']
date_info = date_info[new_column_order]
date_info.head()

## 2. Macroeconomic data

Links to download the csv's:
- GDP Growth: https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG
- GDP: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
- Inflation: https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG
- Population: https://data.worldbank.org/indicator/SP.POP.TOTL

In [ ]:
# Function to skip rows at the header of the csv's
def load_dataset_skiprows(*, file_name: str) -> pd.DataFrame:
    _data = pd.read_csv(file_name, skiprows=4)
    _data= _data.drop_duplicates()
    return _data

In [ ]:
#Define the mapping of countries to their groups
country_to_group = {
    'United States': 'US',

    'Canada': 'Developed',
    'United Kingdom': 'Developed',
    'Germany': 'Developed',
    'France': 'Developed',
    'Switzerland': 'Developed',
    'Netherlands': 'Developed',
    'Sweden': 'Developed',
    'Denmark': 'Developed',
    'Spain': 'Developed',
    'Japan': 'Developed',
    'South Korea': 'Developed',
    'Australia': 'Developed',
    'New Zealand': 'Developed',
    'Singapore': 'Developed',
    'Hong Kong': 'Developed',

    'China': 'Emerging',
    'India': 'Emerging',
    'Indonesia': 'Emerging',
    'Thailand': 'Emerging',
    'Malaysia': 'Emerging',
    'Phlippines': 'Emerging',
    'Vietnam': 'Emerging',
    'Brazil': 'Emerging',
    'Mexico': 'Emerging',
    'Argentina': 'Emerging',
    'Chile': 'Emerging',
    'Colombia': 'Emerging',
    'Peru': 'Emerging',
    'Poland': 'Emerging',
    'Hungary': 'Emerging',
    'Turkey': 'Emerging',
    'South Africa': 'Emerging',
    'Egypt': 'Emerging',
    'Nigeria': 'Emerging',
    'Saudi Arabia': 'Emerging',
    'United Arab Emirates': 'Emerging',
    'Qatar': 'Emerging'
}

### 2.1. GDP Growth by groups of interest

In [ ]:
gdp_growth_global = load_dataset_skiprows(file_name='GDP_GROWTH_GLOBAL.csv')
selected_columns = ['Country Name'] + [str(year) for year in range(2013, 2024)]
gdp_growth_global = gdp_growth_global[selected_columns]

gdp_global = load_dataset_skiprows(file_name='GDP_GLOBAL.csv')
selected_columns = ['Country Name'] + [str(year) for year in range(2013, 2024)]
gdp_global = gdp_global[selected_columns]

In [ ]:
gdp_growth_long = gdp_growth_global.melt(id_vars=['Country Name'], var_name='Year', value_name='GDP growth')
gdp_long = gdp_global.melt(id_vars=['Country Name'], var_name='Year', value_name='GDP')

merged_df = pd.merge(gdp_growth_long, gdp_long, on=['Country Name', 'Year'])
merged_df['Group'] = merged_df['Country Name'].map(country_to_group)

def weighted_avg(df, weight_column, value_column):
    return (df[weight_column] * df[value_column]).sum() / df[weight_column].sum()

grouped_df_growth = merged_df.groupby(['Year', 'Group']).apply(weighted_avg, 'GDP', 'GDP growth').reset_index(name='GDP_Growth')

pivoted_gdp = grouped_df_growth.pivot(index='Year', columns='Group', values='GDP_Growth')
pivoted_gdp.columns = ['GDP_growth_developed', 'GDP_growth_emerging', 'GDP_growth_us']
pivoted_gdp = pivoted_gdp.reset_index()
pivoted_gdp.head()

### 2.2. Inflation by groups of interest

In [ ]:
inflation_global = load_dataset_skiprows(file_name='INFLATION_GLOBAL.csv')
selected_columns = ['Country Name'] + [str(year) for year in range(2013, 2024)]
inflation_global = inflation_global[selected_columns]

population_global = load_dataset_skiprows(file_name='POPULATION_GLOBAL.csv')
selected_columns = ['Country Name'] + [str(year) for year in range(2013, 2024)]
population_global = population_global[selected_columns]

In [ ]:
inflation_long = inflation_global.melt(id_vars=['Country Name'], var_name='Year', value_name='Inflation')
population_long = population_global.melt(id_vars=['Country Name'], var_name='Year', value_name='Population')

merged_df = pd.merge(inflation_long, population_long, on=['Country Name', 'Year'])
merged_df['Group'] = merged_df['Country Name'].map(country_to_group)

def weighted_avg(df, weight_column, value_column):
    return (df[weight_column] * df[value_column]).sum() / df[weight_column].sum()

grouped_inflation_df = merged_df.groupby(['Year', 'Group']).apply(weighted_avg, 'Population', 'Inflation').reset_index(name='Inflation')

pivoted_inflation = grouped_inflation_df.pivot(index='Year', columns='Group', values='Inflation')
pivoted_inflation.columns = ['inflation_developed', 'inflation_emerging', 'inflation_us']
pivoted_inflation = pivoted_inflation.reset_index()
pivoted_inflation.head()

### 2.3 Join info and save

In [ ]:
date_info['Year'] = pd.to_datetime(date_info['datadate']).dt.year

macroeconomic_data = pd.merge(pivoted_gdp, pivoted_inflation, on = ['Year'], how = 'left')
macroeconomic_data['Year'] = macroeconomic_data['Year'].astype(int)
all_info = pd.merge(date_info, macroeconomic_data, on='Year', how='left')
all_info = all_info.drop(columns=['Year'], axis=1)
all_info.head()

In [ ]:
all_info.shape

In [ ]:
all_info.to_csv("datos_1d.csv", index=False)